<a href="https://colab.research.google.com/github/q-vietle/simulation/blob/main/computation_for_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem 2**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
'''data processing'''
import statsmodels.tsa.stattools as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

df = np.loadtxt(fname='', dtype='float')
time = np.arange(df.shape[0])

In [ ]:
'''remove data trend'''

def poly_fit(x,y,degree):
  designed_matrix = sm.add_constant(np.vander(x, degree+1, increasing= False)[:,:-1])
  model = sm.OLS(y, designed_matrix)
  results = model.fit()
  trend = results.predict(designed_matrix)
  residual = y - trend
  trend_model = results.params
  return residual, trend, trend_model

'''main'''
deg =
residual, trend, trend_model = poly_fit(time,df,deg)

In [ ]:
'''find out cut-off'''

import scipy.stats as stats
def z_crit(alpha_level = 0.5):
  z = stats.norm.ppf(1 - alpha_level/2)
  return z
alpha_level =
z_crit = z_crit(alpha_level)

**Bartlett formula**

AR
W[lag,lag] = (1-phi^(2*lag))*(1+phi^2)/(1-phi^2)-2*lag*phi^(2*lag)

MA
W[lag,lag] = sum(rho(i+lag)+rho(lag-i)-2rho(lag)*rho(i))^2 for i from 0 to inf

In [ ]:
'''MA hypothesis'''

n = residual.shape[0]
def hypothesis_for_MA(residual, lag = 1, order = 1):
  acf = sm.tsa.acf(residual, nlags=40)
  W = #bartlett formula
  T_test = (W/n)**(-1/2)*acf[lag]
  p_value = stats.norm.cdf(T_test)
  lb,ub = acf[lag]-z_crit*(W/n)**(1/2),acf[lag]+z_crit*(W/n)**(1/2)
  return T_test,p_value,lb,ub
lag =
order =
T_test,p_value,lb,ub = hypothesis_for_MA(residual, lag, order)
print(f'confidence interval at lag {lag}',(lb,ub))

In [ ]:
'''AR hypothesis'''

n = residual.shape[0]
def hypothesis_for_AR(residual, lag = 1, order = 1):
  acf = sm.tsa.acf(residual, nlags=40)
  phi = acf[1]
  W = (1-phi**(2*lag))*(1+phi**2)/(1-phi**2)-2*lag*phi**(2*lag)#bartlett formula
  T_test = (W/n)**(-1/2)*(acf[lag]-acf[1]**lag)
  p_value = stats.norm.cdf(T_test)
  lb,ub = acf[lag]-acf[1]**lag-z_crit*(W/n)**(1/2),acf[lag]-acf[1]**lag+z_crit*(W/n)**(1/2)
  return T_test,p_value,lb,ub


lag =
order =
T_test,p_value,lb,ub = hypothesis_for_AR(residual, lag, order)
print(f'confidence interval at lag {lag}',(lb,ub))

**Problem 4**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
'''data processing'''

import numpy as np

df = np.loadtxt(fname='', dtype='float')
time = np.arange(df.shape[0])

In [ ]:
'''remove multidimensional data trend '''

def poly_fit(time, data, degree):
    residuals = []
    trends = []
    for i in range(data.shape[1]):
        y = data[:, i]
        X = np.vander(time, degree+1, increasing=False)[:, :-1]
        X = np.concatenate((np.ones((X.shape[0],1)), X), axis=1)
        beta = np.linalg.inv(X.T @ X) @ X.T @ y
        trend = X @ beta
        residual = y - trend
        residuals.append(residual)
        trends.append(trend)
    return np.column_stack(residuals), np.column_stack(trends)

'''main'''
time = time
data = df
degree =
residuals, trends = poly_fit(time, data, degree)

In [ ]:
'''Log likelihood'''

from statsmodels.tsa.statespace.varmax import VARMAX

def ARMA_logLikelihood(residuals, order = (1,1)):
  model = VARMAX(residuals, order = order, trend = None)
  results = model.fit(maxiter=100)
  logL = model.loglike(results.params)
  return logL


In [ ]:
'''Information criteria'''

def ARMA_IC(residuals, order=(1,1)):
    logL = ARMA_logLikelihood(residuals, order = order)
    p = order[0]
    q = order[1]
    n = len(residuals)
    d = residuals.shape[1]
    k = d**2*(p+q)+d*(d+1)/2
    AIC = -2*logL+ 2*k
    AIC_c = -2*logL+k*n/(n-k-1)
    BIC = -2*logL+k*np.log(n)
    return AIC, AIC_c, BIC

def print_IC(residuals, p=[], q=[]):
    for i in p:
        for j in q:
            order = (i,j)
            AIC, AICc, BIC = ARMA_IC(residuals, order=order)
            print(f'AIC, AICc, BIC of ARMA({i},{j}) are respectively', AIC, AICc, BIC)


'''main'''
p =
q =
print_IC(residuals, p, q)

In [ ]:
'''fit data into ARMA by VARMAX'''
p =
q =
steps =
model = VARMAX(residuals, order = (p,q), trend = None)
results = model.fit(maxiter=100)
next_residual = results.forecast(steps = steps)
print('the next residual is',next_residual)

model = VARMAX(df, order = (p,q), trend = None)
results = model.fit(maxiter=100)
next_value = results.forecast(steps = steps)
print('the next value is', next_value)

**Problem 6**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
'''data processing'''

import statsmodels.tsa.stattools as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
df = np.loadtxt(fname='', dtype='float')
time = np.arange(df.shape[0])

In [ ]:
'''remove data trend'''

def poly_fit(x,y,degree):
  designed_matrix = sm.add_constant(np.vander(x, degree+1, increasing= False)[:,:-1])
  model = sm.OLS(y, designed_matrix)
  results = model.fit()
  trend = results.predict(designed_matrix)
  residual = y - trend
  trend_model = results.params
  return residual, trend, trend_model

'''main'''
deg =
residual, trend, trend_model = poly_fit(time,df,deg)

In [ ]:
'''periodogram'''

from scipy.signal import periodogram
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
freq, per = periodogram(residual)
ax.plot(freq,per)


In [ ]:
'''dominance frequency'''

dom_freq = freq[np.argmax(per)]
print(dom_freq)

In [ ]:
'''remove dominant frequency by MA filter'''

from scipy.signal import convolve
def periodogram_ma_filter(residual):
  freq, per = periodogram(residual)
  dom_freq = freq[np.argmax(per)]
  dom_per = round(1/dom_freq)
  ma_filter = np.ones(dom_per)*dom_freq
  ma_filtered = convolve(residual, ma_filter, mode = 'same')
  freq, per = periodogram(ma_filtered)
  return ma_filtered, freq, per

'''main'''
residual_ma_filtered, ma_freq, ma_per = periodogram_ma_filter(residual)
ax.plot(ma_freq,ma_per)


In [ ]:
'''find coefficients of low-pass filter and low_pass_filter'''

def low_pass_filter(num_of_coeffs, cut_off = float):
  coeffs = []
  num_of_coeffs = int(num_of_coeffs)
  if num_of_coeffs == 1:
    coeffs.append(cut_off/np.pi)
  else:
    coeffs.append(cut_off/np.pi)
    for i in range(num_of_coeffs):
      coeffs.append(np.sin((i+1)*cut_off)/((i+1)*np.pi))
  coeffs = np.array(coeffs)
  coeffs = np.concatenate((coeffs[::-1], coeffs[1:]))
  return coeffs

def periodogram_low_pass_filter(residual,size = int, cut_off = float):
  lp_filter = low_pass_filter((size-1)/2, cut_off)
  lp_filtered = convolve(residual, lp_filter, mode ='same')
  freq, per = periodogram(lp_filtered)
  return lp_filtered, freq, per

'''main'''
number_of_coeffs =
size = #odd number
desired_period =
desired_freq = 1/desired_period
cut_off = 2*np.pi*desired_freq

print(low_pass_filter(number_of_coeffs, cut_off))
residual_lp_filtered, lp_freq, lp_per = periodogram_low_pass_filter(residual, size, cut_off)

ax.plot(lp_freq,lp_per)

In [ ]:
'''forecast residuals by ARIMA'''

import statsmodels.tsa.arima.model as ARIMA

def forecast_residual(residual, num_of_next_residual = 1, order = (1, 0, 1)):
    model = sm.tsa.ARIMA(residual, order=order, trend = None)
    results = model.fit()
    next_residual = results.forecast(steps = num_of_next_residual)
    return next_residual

'''main'''

p =
q =
num_of_next_residual =
order = (p, 0, q)

predicted_residuals = forecast_residual(residual, num_of_next_residual, order)

plt.figure()
plt.plot(np.append(residual,predicted_residuals)[-100:])
plt.plot()

In [ ]:
'''forecast values by ARIMA'''

def forecast_value(trend_model,residual, num_of_next_residual = 1, order = (1, 0, 1)):
  predicted_residual = forecast_residual(residual, num_of_next_residual, order)
  trend_next = np.array([num_of_next_residual,residual.shape[0]]) @ trend_model
  predicted_value = trend_next + predicted_residual
  return predicted_value

'''main'''

p =
q =
num_of_next_residual =
order = (p, 0, q)

predicted_values = forecast_value(trend_model,residual, num_of_next_residual, order)

plt.figure()
plt.plot(np.append(df,predicted_values)[-100:])
plt.plot()